In [2]:
import os,sys

In [6]:
print(os.chdir('FinalProject/src/'))

None


In [7]:
os.getcwd()

'/home/ubuntu/Projet/FinalProject/src'

In [ ]:
# %load ModelModule.py
from sklearn.utils import shuffle
from DLModule import DLClass
from DataModule import DataClass
from word2vec_module import MyWord2vec
from loggerModule import LoggerClass
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score as precision
from sklearn.metrics import recall_score as recall
from sklearn.metrics import f1_score
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import load_model


class Model(object):
    def __init__(self, preprocessed_data):
        """
        Init model for training
        :param preprocessed_data: data that was already preprocessed
        """
        self.logger = LoggerClass(name="auto_de")
        self.data = preprocessed_data
        if self.data.preprocessed is not True:
            raise ValueError
        self.nnmodel = None

    def train(self, model_name, model_type, test_size=0.33, **kwargs):
        """
        get all the data preprocess it and train the model and save
        :param model_name: Name of the model for saving
        :param model_type: Type of the model, can be cnn or cblstm
        """
        x, y = shuffle(self.data.preprocess["X"], self.data.labels, random_state=0)
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=42)
        self.nnmodel = DLClass(kwargs=kwargs)
        self.nnmodel.build_model(X_train, y_train, model_type)
        predicts = np.array([i[0] for i in self.nnmodel.model.predict_classes(X_test)])
        print(classification_report(y_test, predicts))
        self.logger.logger.critical("Classification Report: \n{0}".format(classification_report(y_test, predicts)))
        self.nnmodel.model.save("../model/{0}.model".format(model_name))
        self.logger.logger.critical("10 trainings average score START")

    def train_10_avg_score(self, model_name, model_type,  **kwargs):
        self.logger.logger.critical("10 trainings of {0} average score START".format(model_name))
        x, y = shuffle(self.data.preprocess["X"], self.data.labels, random_state=0)
        seed = 7
        n_splits = 10
        kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

        scores = defaultdict(int)
        for train, test in kfold.split(x, y):
            self.nnmodel = DLClass(kwargs=kwargs)
            self.nnmodel.build_model(x[train], y[train], model_type)
            # nnmodel.fit(X_wcl_enriched[train],y_wcl[train],epochs=epochs,batch_size=100)
            print('Predicting...')
            preds = np.array([i[0] for i in nnmodel.predict_classes(x[test])])
            p = precision(preds, y[test])
            r = recall(preds, y[test])
            f1 = f1_score(preds, y[test])
            print('(Fold) Precision: ', p, ' | Recall: ', r, ' | F: ', f1)
            scores['Precision'] += p
            scores['Recall'] += r
            scores['F1'] += f1

        print('Overall scores for model {0}:'.format(model_name))
        for n, sc in scores.items():
            print(n, '-> ', sc / n_splits * 1.0)

    # def predict(self, model_name, data_type=None, my_list=None, threshold=0.8):
    #     """
    #     take the data from my_list , create data object , preprocess it and predict with our model
    #     :param my_list:  data from user
    #     :param data_type: data from dataset
    #     :param model_name: Name of the model to be used for prediction
    #     :param threshold: default 0.8
    #     """
    #     self.data = DataClass(config=self.logger.config)
    #     if my_list is not None:
    #         self.data.data_init(my_list=my_list)
    #     elif data_type is not None:
    #         self.data.data_init(data_type=data_type)
    #     self.preprocess = PreprocessClass(get_data=self.data, get_word2vec=self.word2vec, config=self.logger.config)
    #     self.preprocess.load_all(model_name)
    #     self.preprocess.preprocessing_data()
    #     self.nnmodel = load_model("../model/{0}.model".format(model_name))
    #     for idx, sent in enumerate(self.data.instances):
    #         preds = self.nnmodel.predict(np.array([self.preprocess.X[idx]]))[0][0]
    #         if preds > threshold:
    #             print('{0} Sent: '.format(idx), sent, ' -> ', preds)


def run_module():
    word2vec = MyWord2vec()
    data_wcl = DataClass("wcl", depth="ml")
    data_wcl.getMaxLength(save_stats=True)
    data_wcl.preprocessing_data(word2vec)
    model_wcl = Model(data_wcl)
    model_wcl.train("wcl_cblstm_ml", "cblstm", test_size=0.33)
    model_wcl.train_10_avg_score("wcl_cblstm_ml", "cblstm")

    data_wcl = DataClass("wcl", depth="ml")
    data_wcl.getMaxLength(save_stats=True)
    data_wcl.preprocessing_data(word2vec)
    model_wcl = Model(data_wcl)
    model_wcl.train("wcl_cnn_ml", "cnn", test_size=0.33)
    model_wcl.train_10_avg_score("wcl_cnn_ml", "cnn")

    data_wcl = DataClass("wcl", depth="m")
    data_wcl.getMaxLength(save_stats=True)
    data_wcl.preprocessing_data(word2vec)
    model_wcl = Model(data_wcl)
    model_wcl.train("wcl_cblstm_m", "cblstm", test_size=0.33)
    model_wcl.train_10_avg_score("wcl_cblstm_m", "cblstm")

    data_wcl = DataClass("wcl", depth="m")
    data_wcl.getMaxLength(save_stats=True)
    data_wcl.preprocessing_data(word2vec)
    model_wcl = Model(data_wcl)
    model_wcl.train("wcl_cnn_m", "cnn", test_size=0.33)
    model_wcl.train_10_avg_score("wcl_cnn_m", "cnn")

    data_wcl = DataClass("wcl", depth="")
    data_wcl.getMaxLength(save_stats=True)
    data_wcl.preprocessing_data(word2vec)
    model_wcl = Model(data_wcl)
    model_wcl.train("wcl_cblstm", "cblstm", test_size=0.33)
    model_wcl.train_10_avg_score("wcl_cblstm", "cblstm")

    data_wcl = DataClass("wcl", depth="")
    data_wcl.getMaxLength(save_stats=True)
    data_wcl.preprocessing_data(word2vec)
    model_wcl = Model(data_wcl)
    model_wcl.train("wcl_cnn", "cnn", test_size=0.33)
    model_wcl.train_10_avg_score("wcl_cnn", "cnn")


    # data_wcl = None
    # model_wcl = None
    # import gc
    # gc.collect()
    # data_w00 = DataClass("w00", depth="ml")
    # data_w00.getMaxLength(save_stats=True)
    # data_w00.preprocessing_data(word2vec)
    # model_w00 = Model(data_w00)
    # model_w00.train("w00", "cblstm", test_size=0.33)
    # data_w00 = None
    # model_w00 = None
    # gc.collect()
    # data_wolfram = DataClass("wolfram", depth="ml")
    # data_wolfram.getMaxLength(save_stats=True)
    # data_wolfram.preprocessing_data(word2vec)
    # model_wolfram = Model(data_wolfram)
    # model_wolfram.train("wolfram", "cblstm", test_size=0.33)
    # data_wolfram = None
    # model_wolfram = None
    #gc.collect()
    # data_all = DataClass("", depth="ml")
    # data_all.getMaxLength(save_stats=True)
    # data_all.preprocessing_data(word2vec)
    # model_all = Model(data_all)
    # model_all.train("all", "cblstm", test_size=0.33)


if __name__ == "__main__":
    run_module()



In [ ]:
# %load DLModule.py
import sys
import logging
from keras.models import Sequential
from keras.layers import Dense, Activation, Bidirectional, Dropout, Conv1D, MaxPooling1D, Flatten, LSTM

module_logger = logging.getLogger('auto_de.DLModule')


class DLClass(object):
    def __init__(self,
                 kernel_size=3,
                 filters=100,
                 pool_size=4,
                 strides=1,
                 epochs=10,
                 batch_size=100,
                 Conv1D_padding='valid',
                 Conv1D_activation='relu',
                 dropout=0.5,
                 kwargs={}):
        """
        Init Model base params
        :param kernel_size:
        :param filters:
        :param pool_size:
        :param strides:
        :param epochs:
        :param batch_size:
        :param Conv1D_padding:
        :param Conv1D_activation:
        :param dropout:
        """
        self.logger = logging.getLogger("DLClass")
        self.logger.critical("Build model init")
        self.kernel_size = kernel_size
        self.filters = filters
        self.pool_size = pool_size
        self.strides = strides
        self.epochs = epochs
        self.batch_size = batch_size
        self.Conv1D_padding = Conv1D_padding
        self.Conv1D_activation = Conv1D_activation
        self.dropout = dropout
        self.model = Sequential()
        self.logger.critical("Build model init finished")

    def build_model(self, x, y, model_type, lstm_units=100, validation_data=''):
        """
        Train model
        :param x: preprocessed instances
        :param y: array of labels
        :param model_type: type of model, can be cnn or cblstm
        :param lstm_units: lstm units, default value 100
        :param validation_data:
        """
        self.logger.critical("Build {0} model start".format(model_type))
        self.model.add(Conv1D(self.filters,
                              self.kernel_size,
                              padding=self.Conv1D_padding,
                              activation=self.Conv1D_activation,
                              strides=self.strides,
                              input_shape=(x.shape[1], x.shape[2])))
        self.model.add(MaxPooling1D(pool_size=self.pool_size))
        if model_type == 'cnn':
            self.model.add(Flatten())
            self.model.add(Dropout(self.dropout))
        elif model_type == 'cblstm':
            self.model.add(Bidirectional(LSTM(lstm_units)))
            self.model.add(Dropout(self.dropout))
        else:
            sys.exit('Model type must be "cnn" or "blstm"')
        self.model.add(Dense(1))
        self.model.add(Activation('sigmoid'))
        self.model.compile(loss='binary_crossentropy',
                           optimizer='adam',
                           metrics=['accuracy'])
        print('Train with ', len(x))
        print(self.model.summary())
        self.logger.critical("Model summary\n{0}".format(self.model.summary()))
        self.model.fit(x, y, epochs=self.epochs, batch_size=self.batch_size, validation_data=validation_data)
        self.logger.critical("Build {0} model finished".format(model_type))


In [ ]:
# %load word2vec_module.py
import configparser
import gensim
import pickle
import logging
import sys

# logger for word2vec_module
module_logger = logging.getLogger('auto_de.word2vecModule')
# SECTION NAME in config file
WORD2VEC_CONFIG_SECTION = "word2vec"


class MyWord2vec(object):
    def __init__(self, manual_path_name=None, train_temp_word2vec = False):
        """
        Class that hold the word2vec models from config file or manual input
        :param manual_path_name: Manual option for using another(not default) word2vec model
        :param train_temp_word2vec: Create word2vec for tests
        """
        self.logger = logging.getLogger('auto_de.word2vecModule.MyWord2vec')
        self.config = None
        self.logger.info("Reading configuration from config file")
        self.__init_config()
        if manual_path_name is not None:
            self.path_and_name = manual_path_name
        else:
            if self.config[WORD2VEC_CONFIG_SECTION]["path"][-1:] == '/':
                self.path_and_name = "{0}{1}".format(self.config[WORD2VEC_CONFIG_SECTION]["path"],
                                                     self.config[WORD2VEC_CONFIG_SECTION]["name"])
            else:
                self.path_and_name = "{0}/{1}".format(self.config[WORD2VEC_CONFIG_SECTION]["path"],
                                                      self.config[WORD2VEC_CONFIG_SECTION]["name"])
        self.logger.critical("init word2vec {0}".format(self.path_and_name))
        self.model = None
        self.vocab = None
        self.dims = None
        if not train_temp_word2vec:
            self.__load_embeddings()

    def __load_embeddings(self):
        """
        Method that trying to load the word2vec in different formats
        :return: init self.vocab , self.dims , self.model
        """
        try:
            self.logger.info("Loading word2vec from path {0}".format(self.path_and_name))
            self.model = gensim.models.Word2Vec.load(self.path_and_name)
        except:
            try:
                self.logger.info("load failed")
                self.model = gensim.models.KeyedVectors.load_word2vec_format(self.path_and_name)
            except:
                try:
                    self.logger.info("load_word2vec_format failed")
                    self.model = gensim.models.KeyedVectors.load_word2vec_format(self.path_and_name, binary=True)
                except:
                    try:
                        self.logger.info("load_word2vec_format_binary failed")
                        with open(self.path_and_name, 'rb') as handle:
                            self.model = pickle.load(handle)
                    except:
                        self.logger.error("Couldn't load pickle")
                        raise Exception
        self.logger.critical("load word2vec succeeded")
        try:
            self.vocab = self.model.index2word
        except AttributeError:
            try:
                self.vocab = self.model.wv.index2word
            except:
                self.logger.error("Can't create vocab")
                sys.exit()
        self.logger.info("vocab init succeeded")
        self.dims = self.model.__getitem__(self.vocab[0]).shape[0]
        self.logger.info("dims init succeeded")
        self.vocab = set(self.vocab)

    def __init_config(self, path="configuration/config.ini"):
        """
        Read config file
        :param path: path to config file
        :return: None
        """
        self.config = configparser.ConfigParser()
        self.config.read(path)

    @staticmethod
    def train_word_2_vec():
        """
        Class that train word2vec on the instances from DataClass
        :return: Save the model as a pickle
        """
        from src.DataModule import DataClass
        data_all = DataClass("")
        sentences = data_all.word2vec_temp()
        model = gensim.models.Word2Vec(sentences, min_count=1)
        with open('../bin/word2vec.pickle', 'wb') as handle:
            pickle.dump(model, handle, protocol=pickle.HIGHEST_PROTOCOL)


if __name__ == "__main__":
    # logger = LoggerClass()
    word2vec = MyWord2vec()
    pass



In [ ]:
# %load DataModule.py
import configparser
import ctypes
import json
import os
import pickle
import numpy as np
import logging
import spacy
import gc
from stanfordcorenlp import StanfordCoreNLP
# TODO delete after tests
from word2vec_module import MyWord2vec
from loggerModule import LoggerClass
import pdb
# logger for DataModule
module_logger = logging.getLogger('auto_de.DataModule')
DATA_MODULE_CONFIG_SECTION = "DataModule"


class PyObject(ctypes.Structure):
    _fields_ = [("refcnt", ctypes.c_long)]


class DataClass(object):
    def __init__(self, name, depth="ml"):
        """
        Class that contains data for model training \n
        Config file location is /src/configuration/config.ini \n
        :param name: name of the dataset to be loaded valid_data_names = {'wcl', 'w00', 'wolfram', ""}
        :param depth: The depth in the training process, can be ml ,m, or empty string
        """
        self.logger = logging.getLogger('auto_de.DataModule.DataClass')
        self.logger.info("Data Class init Start")
        self.name = name
        self.preprocessed = False
        self.instances = []
        self.labels = np.array([])
        self.config = None
        self.__init_config()
        self.init_data(name)
        self.preprocess = None
        self.__init_preprocess(depth)
        self.logger.info("Data Class init Finished")

    def __init_config(self, path="configuration/config.ini"):
        """
        Read config file
        :param path: path to config file
        :return: None
        """
        self.config = configparser.ConfigParser()
        self.config.read(path)

    def init_data(self, name):
        """
        Load specific data set to DataCLass object
        :param name: name of the dataset valid_data_names = {'wcl', 'w00', 'wolfram', ""}
        :return: None
        """
        folder = None
        valid_data_names = {'wcl', 'w00', 'wolfram', ""}
        path = self.config[DATA_MODULE_CONFIG_SECTION]["path"]
        if name not in valid_data_names:
            self.logger.error("results: status must be one of %r." % valid_data_names)
            raise ValueError("results: status must be one of %r." % valid_data_names)
        else:
            self.logger.critical("init {0}{1} dataset".format(path, name))
        if name is not "":
            folder = self.config[DATA_MODULE_CONFIG_SECTION][name]
        if name == "wcl":
            self.__load_wcl(path, folder)
        elif name == "w00":
            self.__load_w00(path, folder)
        elif name == "wolfram":
            self.__load_wolfram(path, folder)
        else:
            self.__load_all(path)
        self.logger.info("Reading data form file succeeded")

    # TODO delete this if no need
    # def load_data_from_file(self, data_type=0):
    #     """
    #     Function that load data form file and send it to different parse functions like wcl, w00, wolfram
    #     :param data_type:0 if all data , 1 if wcl 2 if w00 3 if wolfram
    #     :return:None just save the data in the self.instances and self.labels
    #     """
    #     sents = None
    #     self.logger.info("Reading Data from file")
    #     path = self.config[CONFIG_SELECTION][DATA_SETS_LIST[data_type]]
    #     self.logger.info("Config -> Data -> Path -> {0}".format(path))
    #     for root, subdirs, files in os.walk(path):
    #         for filename in files:
    #             if filename.startswith("annotated"):
    #                 self.logger.info("Reading w00 data")
    #                 if filename == 'annotated.word':
    #                     sents = open(os.path.join(root, filename), 'r').readlines()
    #                 elif filename == 'annotated.meta':
    #                     labels = open(os.path.join(root, filename), 'r').readlines()
    #                 if sents and labels:
    #                     for idx, sent in enumerate(sents):
    #                         sent = sent.strip().lower()
    #                         label = int(labels[idx].split(' $ ')[0])
    #                         self.instances.append(sent)
    #                         self.labels = np.concatenate((self.labels, np.array(label)), axis=None)
    #             if filename.startswith("wolfram"):
    #                 label = filename.split('_')[-1].replace('.txt', '')
    #                 doc = os.path.join(root, filename)
    #                 lines = open(doc, 'r', encoding='utf-8').readlines()
    #                 try:
    #                     self.logger.info("Start pars_wolfram")
    #                     labels, instances = self.__pars_wolfram(label, lines)
    #                 except:
    #                     self.logger.error("__pars_wolfram failed")
    #                     raise
    #                 self.logger.info("__pars_wolfram succeeded")
    #                 self.instances = self.instances + instances
    #                 self.labels = np.concatenate((self.labels, np.array(labels)), axis=None)
    #
    #     self.logger.info("Reading data form file succeeded")

    @staticmethod
    def __pars_wcl(label, lines):
        """
        Function that parse the wcl data lines to a separated instances and append there real label
        :param label: good or bad
        :param lines: all the data from file by lines
        :return: array of labels and instances for each instances : instances[i] have a classification labels[i]
        """
        labels, instances = [], []
        for idx, line in enumerate(lines):
            if line.startswith('#'):
                target = lines[idx + 1].split(':')[0]
                if target[0] == "!":
                    target = target[1:]
                sent = line[2:].replace('TARGET', target).strip().lower()
                if label == 'good':
                    labels.append(1)
                else:
                    labels.append(0)
                instances.append(sent)
        return labels, instances

    # TODO  delete this if no need
    # def save_bin_data(self):
    #     """
    #     Save data to pickle(bin) file
    #     :return: None just save to file .pickle the instances and the labels
    #     """
    #     name = self.config["BIN"]["data_save"]
    #     self.logger.info("Saving data to bin with name {0}".format(name))
    #     with open('../bin/{0}Instances.pickle'.format(name), 'wb') as handle:
    #         pickle.dump(self.instances, handle, protocol=pickle.HIGHEST_PROTOCOL)
    #     self.logger.info("Instances Data saved to {0}Instances.pickle".format(name))
    #     with open('../bin/{0}Labels.pickle'.format(name), 'wb') as handle:
    #         pickle.dump(self.labels, handle, protocol=pickle.HIGHEST_PROTOCOL)
    #     self.logger.info("Labels Data saved to {0}Labels.pickle".format(name))
    #
    # def load_bin_data(self, name="Data"):
    #     """
    #     Load Data from a pickle(bin) to save time for parsing or using different data
    #     :param name: If you want to save to a different file and not the default one
    #     :return: save the data to self.instances and self.labels
    #     """
    #     with open('../bin/{0}Instances.pickle'.format(name), 'rb') as handle:
    #         self.instances = pickle.load(handle)
    #
    #     with open('../bin/{0}Labels.pickle'.format(name), 'rb') as handle:
    #         self.labels = pickle.load(handle)

    # TODO delete after tests
    def word2vec_temp(self):
        """
        Method that create structure for word2vec training
        Used for tem word2vec creation for debugging
        :return: [["word1","word2","word3"...],[sentence 2],[sentence 3]...]
        """
        nlp = spacy.load("en_core_web_sm")
        result_main = []
        for line in self.instances:
            sentenses = nlp(line)
            result = []
            for idx, token in enumerate(sentenses):
                result.append(token.orth_)
            result_main.append(result)
        return result_main

    # TODO delete this if no need
    # def data_init(self, my_list=None, data_type=0):
    #     """
    #     Method that load data to data class fro training and prediction
    #     list option for api use (don't init labels)
    #     :param my_list: List of lists of sentences
    #     :param data_type: 0 - Get all the data from data folder
    #                       1 - get wcl dataset only
    #                       2 - get w00 dataset only
    #
    #     :return:init self.instances and self.labels if not list option
    #     """
    #     if self.config["DataModule"]["get"] == 'bin':
    #         name = self.config["BIN"]["data_load"]
    #         self.logger.info("Load data form bin file {0}".format(name))
    #         self.load_bin_data(name=name)
    #         self.logger.info("Load data from bin finished")
    #     elif self.config["DataModule"]["get"] == 'file':
    #         self.logger.info("Load data form file {0}".format(data_type))
    #         self.load_data_from_file(data_type=data_type)
    #         self.logger.info("Load data from file finished")
    #     elif self.config["DataModule"]["get"] == 'list' and my_list is not None \
    #             and not self.config.getboolean("DEFAULT", "Train"):
    #         self.logger.info("Reading data from list")
    #         for instance in my_list:
    #             self.instances.append(instance)
    #         self.logger.info("Reading data from list finished")
    #     else:
    #         self.logger.error("The [get] option {0} in the config file isn't known!!!"
    #                           .format(self.config["DataModule"]["get"]))
    #         raise Exception

    @staticmethod
    def __pars_wolfram(label, lines):
        """
        Function that parse the w00 data lines to a separated instances and append there real label
        :param label: good or bad
        :param lines: all the data from file by lines
        :return: array of labels and instances for each instances : instances[i] have a classification labels[i]
        """
        labels, instances = [], []
        for idx, line in enumerate(lines):
            if label == 'good':
                labels.append(1)
            else:
                labels.append(0)
            instances.append(line)
        return labels, instances

    def __load_wcl(self, path, folder):
        """
        Load wcl dataset
        :param path: path to data folder
        :param folder: dataset folder name
        :return: init self.instances and self.labels
        """
        self.logger.critical("Reading wiki data")
        for root, subdirs, files in os.walk(path+folder):
            for filename in files:
                if filename.startswith('wiki_'):
                    label = filename.split('_')[-1].replace('.txt', '')
                    doc = os.path.join(root, filename)
                    lines = open(doc, 'r', encoding='utf-8').readlines()
                    try:
                        self.logger.info("Start pars_wcl")
                        labels, instances = self.__pars_wcl(label, lines)
                    except:
                        self.logger.error("__pars_wcl failed")
                        raise
                    self.logger.info("__pars_wcl succeeded")
                    self.instances = self.instances + instances
                    self.labels = np.concatenate((self.labels, np.array(labels)), axis=None)

    def __load_w00(self, path, folder):
        """
        Load w00 dataset
        :param path: path to data folder
        :param folder: dataset folder name
        :return: init self.instances and self.labels
        """
        self.logger.critical("Reading w00 data")
        error_count = 0
        for root, subdirs, files in os.walk(path+folder):
            for filename in files:
                if filename.startswith("annotated"):
                    if filename == 'annotated.word':
                        sents = open(os.path.join(root, filename), 'r').readlines()
                    elif filename == 'annotated.meta':
                        labels = open(os.path.join(root, filename), 'r').readlines()
                    try:
                        if (sents is not None) and (labels is not None):
                            for idx, sent in enumerate(sents):
                                sent = sent.strip().lower()
                                label = int(labels[idx].split(' $ ')[0])
                                self.instances.append(sent)
                                self.labels = np.concatenate((self.labels, np.array(label)), axis=None)
                    except UnboundLocalError:
                        if error_count < 10:
                            error_count = error_count+1
                            pass
                        else:
                            raise UnboundLocalError

    def __load_wolfram(self, path, folder):
        """
        Load wolfram dataset
        :param path: path to data folder
        :param folder: dataset folder name
        :return: init self.instances and self.labels
        """
        self.logger.critical("Reading wolfram data")
        for root, subdirs, files in os.walk(path+folder):
            for filename in files:
                if filename.startswith("wolfram"):
                    label = filename.split('_')[-1].replace('.txt', '')
                    doc = os.path.join(root, filename)
                    lines = open(doc, 'r', encoding='utf-8').readlines()
                    try:
                        self.logger.info("Start pars_wolfram")
                        labels, instances = self.__pars_wolfram(label, lines)
                    except:
                        self.logger.error("__pars_wolfram failed")
                        raise
                    self.logger.info("__pars_wolfram succeeded")
                    self.instances = self.instances + instances
                    self.labels = np.concatenate((self.labels, np.array(labels)), axis=None)

    def __load_all(self, path):
        """
        Load all datasets together
        :param path: path to datasets folder
        :return: None
        """
        self.__load_wcl(path, "")
        self.__load_w00(path, "")
        self.__load_wolfram(path, "")

    @staticmethod
    def get_pair_words(json_object):
        """
        Method that pars Stanford core nlp return jason
        :param json_object: json from Stanfor core nlp
        :return: parsed dictionary
        """
        result = {}
        res = {}
        try:
            [res.update({k['dependent']: k['dep']}) for k in json_object['sentences'][0]['basicDependencies']]
        except IndexError:
            raise IndexError
        for idx, k in enumerate(json_object['sentences'][0]['basicDependencies'][1:]):
            result[idx] = {'parent_word': json_object['sentences'][0]['basicDependencies'][1:][idx]['governorGloss'],
                           'parent_dep': res[json_object['sentences'][0]['basicDependencies'][1:][idx]['governor']],
                           'child_word': json_object['sentences'][0]['basicDependencies'][1:][idx]['dependentGloss'],
                           'child_dep': json_object['sentences'][0]['basicDependencies'][1:][idx]['dep']}
        return result

    def __set_depth(self, depth=""):
        """
        Need to be used for different prepossess methods
        :return:
        """
        try:
            if depth == 'ml':
                self.preprocess['X'] = np.concatenate([self.preprocess["X"], self.preprocess["X_deps"]], axis=1)
                self.logger.critical("Setting depth ml")
            elif depth == 'm':
                self.preprocess['X'] = np.concatenate([self.preprocess["X"], self.preprocess['X_wordpairs']], axis=1)
                self.logger.info("Setting depth m")
            else:
                self.logger.info("Setting no depth")
        except Exception:
            self.logger.error("Setting depth failed")
            raise

    def __init_preprocess(self, depth):
        self.preprocess = dict()
        self.preprocess["X"] = []
        self.preprocess["X_deps"] = []
        self.preprocess["X_wordpairs"] = []
        self.preprocess['depth'] = depth

    def __get_stanford_core_nlp(self):
        """
        Load StanfordCoreNlp jar to python wrapper
        :return: stanford core nlp wrapper instance
        """
        return StanfordCoreNLP(self.config[DATA_MODULE_CONFIG_SECTION]['nlp_path'])

    def getMaxLength(self, save_stats=False):
        """
        Get stats from data maxlen, deps2ids, ids2deps
        maxlen - the longest sentence or dependencies length
        :param save_stats: True or false for saving stats to pickle
        :type save_stats: Boolean
        """
        self.logger.critical('Getting maxlen')
        maxlen = 0
        deps2ids = {}
        depid = 0
        maxlen_dep = 0
        nlp = self.__get_stanford_core_nlp()
        for idx, sent in enumerate(self.instances):
            if idx % 20 == 0:
                self.logger.critical('Done {0} of {1}'.format(idx, len(self.instances)))
            try:
                sent_maxlen_dep = 0
                doc = nlp.dependency_parse(sent)
                if len(doc) > maxlen:
                    maxlen = len(doc)
                doc_set = set([k[0] for k in doc])
                for token in doc_set:
                    if token not in deps2ids:
                        deps2ids[token] = depid
                        depid += 1
                        sent_maxlen_dep += 1
                if sent_maxlen_dep > maxlen_dep:
                    maxlen_dep = sent_maxlen_dep
            except UnicodeDecodeError:
                self.logger.error('Cant process sentence: ', sent)
        maxlen = max(maxlen, maxlen_dep)
        ids2deps = dict([(idx, dep) for dep, idx in deps2ids.items()])
        self.preprocess["maxlen"] = maxlen
        self.preprocess["deps2ids"] = deps2ids
        self.preprocess["ids2deps"] = ids2deps
        nlp.close()
        nlp = None
        if save_stats:
            self.save_stats(self.name)
        self.logger.critical('Maxlen = {0}'.format(self.preprocess["maxlen"]))

    @staticmethod
    def save_obj(obj, name, path="../bin/"):
        """
        Static method that can save any object to pickle
        :param obj: The object need to be saved
        :param name: Name of saved file
        :param path: path to save the file
        :return: None
        """
        with open(path + name + '.pkl', 'wb') as f:
            pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

    @staticmethod
    def load_obj(name, path="../bin/"):
        """
        Static method that can load any python object
        :param name: name of the file
        :param path: path to the directory of the object
        :return: object from pickle
        """
        with open(path + name + '.pkl', 'rb') as f:
            return pickle.load(f)

    def save_stats(self, name):
        """
        Method that save to pickle ids2deps,deps2ids and maxlen
        :return: None
        """
        self.logger.critical("Saving stats of {0} to file".format(name))
        self.save_obj(self.preprocess['ids2deps'], "{0}_ids2deps".format(name))
        self.save_obj(self.preprocess['deps2ids'], "{0}_deps2ids".format(name))
        self.save_obj(self.preprocess['maxlen'], "{0}_maxlen".format(name))

    def load_all_stats(self, name):
        """
        Method that load from pickle ids2deps,deps2ids and maxlen
        :return: save the self instances
        """
        self.logger.critical("Load stats {0} from file".format(name))
        self.preprocess['ids2deps'] = self.load_obj("{0}_ids2deps".format(name))
        self.preprocess['deps2ids'] = self.load_obj("{0}_deps2ids".format(name))
        self.preprocess['maxlen'] = self.load_obj("{0}_maxlen".format(name))

    @staticmethod
    def pad_words(tokens, maxlen, append_tuple=False):
        """
        Static Method that is part of the Preprocess
        make the sentences arrays same length
        :param tokens: tokens to be modified
        :param maxlen: max length of the longest sentance or dependency length
        :param append_tuple:  Need to be true when tuple need to be append
        :return: return modified tokens
        """
        if len(tokens) > maxlen:
            return tokens[:maxlen]
        else:
            dif = maxlen - len(tokens)
            for i in range(dif):
                if not append_tuple:
                    tokens.append('UNK')
                else:
                    tokens.append(('UNK', 'UNK'))
            return tokens

    def preprocessing_data(self, word2vec):
        """
        Method that takes all the instances in the data and preprocess them.
        :param word2vec: word2vec that need to be used in the preprocess
        """
        self.logger.critical("Starting to preprocess data {}".format(self.name))
        nlp = self.__get_stanford_core_nlp()
        try:
            self.logger.info("Max length of the data is {0}".format(self.preprocess['maxlen']))
        except KeyError:
            self.load_all_stats(self.name)
        for idx, sent in enumerate(self.instances):
            if idx % 100 == 0:
                self.logger.critical('Words done {0} of  {1}'.format(idx, len(self.instances)))
            object_json_data = json.loads(nlp.annotate(sent, properties={'annotators': 'tokenize', 'outputFormat': 'json'}))
            tokens = [k['word'].lower() for k in object_json_data['tokens']]
            sent_matrix = []
            for token in self.pad_words(tokens, self.preprocess['maxlen']):
                if token in word2vec.vocab:
                    # each word vector is embedding dim + length of one-hot encoded label
                    vec = np.concatenate([word2vec.model[token], np.zeros(len(self.preprocess['ids2deps'])+1)])
                    sent_matrix.append(vec)
                else:
                    sent_matrix.append(np.zeros(word2vec.dims + len(self.preprocess['ids2deps']) + 1))
            sent_matrix = np.array(sent_matrix)
            try:
                self.preprocess['X'].append(sent_matrix)
            except AttributeError:
                np.concatenate([self.preprocess['X'], [sent_matrix]])

        for idx, sent in enumerate(self.instances):
            if idx % 10 == 0:
                self.logger.critical('Pairs done {0} of  {1}'.format(idx, len(self.instances)))
            object_json_data = json.loads(nlp.annotate(sent, properties={'annotators': 'depparse', 'outputFormat': 'json'}))
            try:
                tokens = self.get_pair_words(object_json_data)
            except IndexError:
                pdb.set_trace()
                pass
            word_pairs = []
            dep_pairs = []
            for idx2, tok in tokens.items():
                word_pairs.append((tok['parent_word'], tok['child_word']))
                dep_pairs.append((tok['parent_dep'], tok['child_dep']))
            self.pad_words(word_pairs, self.preprocess['maxlen'], append_tuple=True)
            self.pad_words(dep_pairs, self.preprocess['maxlen'], append_tuple=True)
            dep_labels = [j for i, j in dep_pairs]
            avg_sent_matrix = []
            avg_label_sent_matrix = []
            for idx, word_pair in enumerate(word_pairs):
                head, modifier = word_pair[0], word_pair[1]
                if head in word2vec.vocab and not head == 'UNK':
                    head_vec = word2vec.model[head]
                else:
                    head_vec = np.zeros(word2vec.dims)
                if modifier in word2vec.vocab and not modifier == 'UNK':
                    modifier_vec = word2vec.model[modifier]
                else:
                    modifier_vec = np.zeros(word2vec.dims)
                avg = np.mean(np.array([head_vec, modifier_vec]), axis=0)
                if dep_labels[idx] != 'UNK':
                    try:
                        dep_idx = self.preprocess['deps2ids'][dep_labels[idx]]
                    except KeyError:
                        dep_idx = -1
                else:
                    dep_idx = -1
                dep_vec = np.zeros(len(self.preprocess['deps2ids']) + 1)
                dep_vec[dep_idx] = 1
                avg_label_vec = np.concatenate([avg, dep_vec])
                avg_sent_matrix.append(np.concatenate([avg, np.zeros(len(self.preprocess['deps2ids']) + 1)]))
                avg_label_sent_matrix.append(avg_label_vec)
            wp = np.array(avg_sent_matrix)
            labs = np.array(avg_label_sent_matrix)
            try:
                self.preprocess['X_wordpairs'].append(wp)
            except AttributeError:
                np.concatenate([self.preprocess['X_wordpairs'], [wp]])
            try:
                self.preprocess['X_deps'].append(labs)
            except AttributeError:
                np.concatenate([self.preprocess['X_deps'], [labs]])
        nlp.close()
        nlp_address = id(nlp)
        print("Nlp reference count before: ", PyObject.from_address(nlp_address).refcnt)
        del nlp
        print("Nlp reference count after: ", PyObject.from_address(nlp_address).refcnt)
        word2vec_address = id(word2vec)
        print("word2vec reference count before: ", PyObject.from_address(word2vec_address).refcnt)
        del word2vec
        print("word2vec reference count after: ", PyObject.from_address(word2vec_address).refcnt)
        self.logger.critical('1 Garbage Collector: Collected amount {0}'.format(gc.collect()))
        X_wordpairs_address = id(self.preprocess['X_wordpairs'])
        print("X_wordpairs reference count: ", PyObject.from_address(X_wordpairs_address).refcnt)
        X_deps_address = id(self.preprocess['X_deps'])
        print("X_deps reference count: ", PyObject.from_address(X_deps_address).refcnt)
        instancess_address = id(self.instances)
        print("instances reference count: ", PyObject.from_address(instancess_address).refcnt)
        del self.instances
        try:
            self.logger.critical('2 Garbage Collector: Collected amount {0}'.format(gc.collect()))
            if self.preprocess['X'].__class__ is not np.array([]).__class__:
                self.preprocess['X'] = np.array(self.preprocess['X'])
            if self.preprocess['X_wordpairs'].__class__ is not np.array([]).__class__ and self.preprocess['depth'] == 'm':
                del self.preprocess['X_deps']
                self.preprocess['X_wordpairs'] = np.array(self.preprocess['X_wordpairs'])
            elif self.preprocess['X_deps'].__class__ is not np.array([]).__class__ and self.preprocess['depth'] == 'ml':
                del self.preprocess['X_wordpairs']
                self.preprocess['X_deps'] = np.array(self.preprocess['X_deps'])
            if self.preprocess['depth'] == '':
                del self.preprocess['X_deps']
                del self.preprocess['X_wordpairs']
        except MemoryError:
            pdb.set_trace()
            pass
        self.logger.critical('3 Garbage Collector: Collected amount {0}'.format(gc.collect()))
        self.__set_depth(self.preprocess['depth'])
        self.preprocessed = True
        self.logger.critical("Data prepossess succeeded ")


if __name__ == '__main__':
    logger = LoggerClass()
    word2vec = MyWord2vec()
    data_wcl = DataClass("wcl")
    data_wcl.getMaxLength()
    data_wcl.preprocessing_data(word2vec)
    data_w00 = DataClass("w00")
    data_w00.getMaxLength()
    data_w00.preprocessing_data(word2vec)
    data_wolfram = DataClass("wolfram")
    data_wolfram.getMaxLength()
    data_wolfram.preprocessing_data(word2vec)
    data_all = DataClass("")
    data_all.getMaxLength()
    data_all.preprocessing_data(word2vec)
    pass







In [13]:
word2vec = MyWord2vec()
data_wcl = DataClass("wcl", depth="ml")
data_wcl.getMaxLength(save_stats=True)
data_wcl.preprocessing_data(word2vec)

NameError: name 'MyWord2vec' is not defined

In [15]:
%run ModelModule.py

INFO:auto_de.word2vecModule.MyWord2vec:Reading configuration from config file
2018-09-11 17:17:58,440 - auto_de.word2vecModule.MyWord2vec - CRITICAL - init word2vec ../bin/GoogleNews-vectors-negative300.bin
CRITICAL:auto_de.word2vecModule.MyWord2vec:init word2vec ../bin/GoogleNews-vectors-negative300.bin
INFO:auto_de.word2vecModule.MyWord2vec:Loading word2vec from path ../bin/GoogleNews-vectors-negative300.bin
INFO:auto_de.word2vecModule.MyWord2vec:load failed
INFO:auto_de.word2vecModule.MyWord2vec:load_word2vec_format failed
2018-09-11 17:18:42,962 - auto_de.word2vecModule.MyWord2vec - CRITICAL - load word2vec succeeded
CRITICAL:auto_de.word2vecModule.MyWord2vec:load word2vec succeeded
INFO:auto_de.word2vecModule.MyWord2vec:vocab init succeeded
INFO:auto_de.word2vecModule.MyWord2vec:dims init succeeded
INFO:auto_de.DataModule.DataClass:Data Class init Start
2018-09-11 17:18:43,345 - auto_de.DataModule.DataClass - CRITICAL - init ../data/wcl dataset
CRITICAL:auto_de.DataModule.DataClas

2018-09-11 17:19:15,823 - auto_de.DataModule.DataClass - CRITICAL - Done 940 of 4719
CRITICAL:auto_de.DataModule.DataClass:Done 940 of 4719
2018-09-11 17:19:16,229 - auto_de.DataModule.DataClass - CRITICAL - Done 960 of 4719
CRITICAL:auto_de.DataModule.DataClass:Done 960 of 4719
2018-09-11 17:19:16,673 - auto_de.DataModule.DataClass - CRITICAL - Done 980 of 4719
CRITICAL:auto_de.DataModule.DataClass:Done 980 of 4719
2018-09-11 17:19:17,188 - auto_de.DataModule.DataClass - CRITICAL - Done 1000 of 4719
CRITICAL:auto_de.DataModule.DataClass:Done 1000 of 4719
2018-09-11 17:19:17,665 - auto_de.DataModule.DataClass - CRITICAL - Done 1020 of 4719
CRITICAL:auto_de.DataModule.DataClass:Done 1020 of 4719
2018-09-11 17:19:18,048 - auto_de.DataModule.DataClass - CRITICAL - Done 1040 of 4719
CRITICAL:auto_de.DataModule.DataClass:Done 1040 of 4719
2018-09-11 17:19:18,506 - auto_de.DataModule.DataClass - CRITICAL - Done 1060 of 4719
CRITICAL:auto_de.DataModule.DataClass:Done 1060 of 4719
2018-09-11 1

2018-09-11 17:19:39,430 - auto_de.DataModule.DataClass - CRITICAL - Done 2100 of 4719
CRITICAL:auto_de.DataModule.DataClass:Done 2100 of 4719
2018-09-11 17:19:39,956 - auto_de.DataModule.DataClass - CRITICAL - Done 2120 of 4719
CRITICAL:auto_de.DataModule.DataClass:Done 2120 of 4719
2018-09-11 17:19:40,316 - auto_de.DataModule.DataClass - CRITICAL - Done 2140 of 4719
CRITICAL:auto_de.DataModule.DataClass:Done 2140 of 4719
2018-09-11 17:19:40,732 - auto_de.DataModule.DataClass - CRITICAL - Done 2160 of 4719
CRITICAL:auto_de.DataModule.DataClass:Done 2160 of 4719
2018-09-11 17:19:41,199 - auto_de.DataModule.DataClass - CRITICAL - Done 2180 of 4719
CRITICAL:auto_de.DataModule.DataClass:Done 2180 of 4719
2018-09-11 17:19:41,582 - auto_de.DataModule.DataClass - CRITICAL - Done 2200 of 4719
CRITICAL:auto_de.DataModule.DataClass:Done 2200 of 4719
2018-09-11 17:19:41,897 - auto_de.DataModule.DataClass - CRITICAL - Done 2220 of 4719
CRITICAL:auto_de.DataModule.DataClass:Done 2220 of 4719
2018-0

2018-09-11 17:20:02,488 - auto_de.DataModule.DataClass - CRITICAL - Done 3260 of 4719
CRITICAL:auto_de.DataModule.DataClass:Done 3260 of 4719
2018-09-11 17:20:02,931 - auto_de.DataModule.DataClass - CRITICAL - Done 3280 of 4719
CRITICAL:auto_de.DataModule.DataClass:Done 3280 of 4719
2018-09-11 17:20:03,298 - auto_de.DataModule.DataClass - CRITICAL - Done 3300 of 4719
CRITICAL:auto_de.DataModule.DataClass:Done 3300 of 4719
2018-09-11 17:20:03,777 - auto_de.DataModule.DataClass - CRITICAL - Done 3320 of 4719
CRITICAL:auto_de.DataModule.DataClass:Done 3320 of 4719
2018-09-11 17:20:04,504 - auto_de.DataModule.DataClass - CRITICAL - Done 3340 of 4719
CRITICAL:auto_de.DataModule.DataClass:Done 3340 of 4719
2018-09-11 17:20:04,970 - auto_de.DataModule.DataClass - CRITICAL - Done 3360 of 4719
CRITICAL:auto_de.DataModule.DataClass:Done 3360 of 4719
2018-09-11 17:20:05,430 - auto_de.DataModule.DataClass - CRITICAL - Done 3380 of 4719
CRITICAL:auto_de.DataModule.DataClass:Done 3380 of 4719
2018-0

2018-09-11 17:20:26,887 - auto_de.DataModule.DataClass - CRITICAL - Done 4420 of 4719
CRITICAL:auto_de.DataModule.DataClass:Done 4420 of 4719
2018-09-11 17:20:27,305 - auto_de.DataModule.DataClass - CRITICAL - Done 4440 of 4719
CRITICAL:auto_de.DataModule.DataClass:Done 4440 of 4719
2018-09-11 17:20:27,758 - auto_de.DataModule.DataClass - CRITICAL - Done 4460 of 4719
CRITICAL:auto_de.DataModule.DataClass:Done 4460 of 4719
2018-09-11 17:20:28,143 - auto_de.DataModule.DataClass - CRITICAL - Done 4480 of 4719
CRITICAL:auto_de.DataModule.DataClass:Done 4480 of 4719
2018-09-11 17:20:28,514 - auto_de.DataModule.DataClass - CRITICAL - Done 4500 of 4719
CRITICAL:auto_de.DataModule.DataClass:Done 4500 of 4719
2018-09-11 17:20:28,813 - auto_de.DataModule.DataClass - CRITICAL - Done 4520 of 4719
CRITICAL:auto_de.DataModule.DataClass:Done 4520 of 4719
2018-09-11 17:20:29,222 - auto_de.DataModule.DataClass - CRITICAL - Done 4540 of 4719
CRITICAL:auto_de.DataModule.DataClass:Done 4540 of 4719
2018-0

...

(Fold) Precision:  0.8877005347593583  | Recall:  0.8877005347593583  | F:  0.8877005347593583


CRITICAL:DLClass:Build cnn model start
CRITICAL:DLClass:Model summary
None


Train with  4248
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_68 (Conv1D)           (None, 267, 100)          102400    
_________________________________________________________________
max_pooling1d_68 (MaxPooling (None, 66, 100)           0         
_________________________________________________________________
flatten_33 (Flatten)         (None, 6600)              0         
_________________________________________________________________
dropout_68 (Dropout)         (None, 6600)              0         
_________________________________________________________________
dense_68 (Dense)             (None, 1)                 6601      
_________________________________________________________________
activation_68 (Activation)   (None, 1)                 0         
Total params: 109,001
Trainable params: 109,001
Non-trainable params: 0
_____________________________________________________

CRITICAL:DLClass:Build cnn model finished


Predicting...
(Fold) Precision:  0.8877005347593583  | Recall:  0.9485714285714286  | F:  0.9171270718232044
Overall scores for model wcl_cnn:
Precision ->  0.9091591762430312
Recall ->  0.9146247748068677
F1 ->  0.9113825755121494


# Overall scores for model wcl_cblstm_ml:

Precision ->  0.9332062805779954

Recall ->  0.9231746613886674

F1 ->  0.9278004912466699

# Overall scores for model wcl_cnn_ml:

Precision ->  0.9257338718853111

Recall ->  0.9345650164741259

F1 ->  0.9296423682801634

# Overall scores for model wcl_cblstm_m:

Precision ->  0.9251934235976789

Recall ->  0.9184189807947879

F1 ->  0.9214650618282295

# Overall scores for model wcl_cnn_m:

Precision ->  0.9155933553305268

Recall ->  0.9082715701652377

F1 ->  0.9114203672626691

# Overall scores for model wcl_cblstm:

Precision ->  0.9246871088861074

Recall ->  0.9220586030631808

F1 ->  0.9229475999503617

# Overall scores for model wcl_cnn:

Precision ->  0.9091591762430312

Recall ->  0.9146247748068677

F1 ->  0.9113825755121494